In [52]:
import pandas as pd
import numpy as np
import extract_images
import extract_figure_info

In [53]:
def merge_image_and_figure_info(image_info_csv, input_folder, pattern, output_csv):
    """
    画像情報と図番号情報をマージしてCSVファイルに保存する関数。

    Args:
        image_info_csv (str): 画像情報が格納されたCSVファイルのパス。
        pdf_folder (str): PDFファイルが格納されているフォルダのパス。
        pattern (str): 図番号の正規表現パターン。
        output_csv (str): 出力CSVファイルのパス。
    """
    # 画像情報と図番号情報を含むデータフレームを作成する
    image_info = pd.read_csv(image_info_csv)  # 画像情報が格納されたCSVファイル
    figure_info = extract_figure_info.extract_figure_info_from_folder(pdf_folder, pattern)  # 図番号情報を取得する関数

    # マージしたデータを格納するリストを初期化
    merged_data = []

    for _, img_row in image_info.iterrows():
        img_page = img_row['ページ番号']
        img_x0, img_y0, img_x1, img_y1 = img_row['x0'], img_row['y0'], img_row['x1'], img_row['y1']

        # 一致する図番号情報をフィルタリング
        matching_figures = figure_info[
            (figure_info['PDFファイル'] == img_row['PDFファイル']) &
            (figure_info['ページ番号'] == img_page)
        ]

        if not matching_figures.empty:
            best_match = None
            min_distance = np.inf

            for _, fig_row in matching_figures.iterrows():
                fig_x0, fig_y0, fig_x1, fig_y1 = fig_row['x0'], fig_row['y0'], fig_row['x1'], fig_row['y1']

                # 画像と図番号の座標の差を計算
                distance = np.sqrt((img_x0 - fig_x0)**2 + (img_y0 - fig_y0)**2)

                if distance < min_distance:
                    best_match = fig_row
                    min_distance = distance

            if best_match is not None:
                merged_data.append({
                    'PDFファイル': img_row['PDFファイル'],
                    'ページ番号': img_page,
                    '画像ファイル': img_row['画像ファイル'],
                    '図番号': best_match['図番号'],
                    'x0': img_x0,
                    'y0': img_y0,
                    'x1': img_x1,
                    'y1': img_y1
                })

    # マージしたデータをデータフレームに変換
    merged_df = pd.DataFrame(merged_data)

    # マージしたデータフレームを保存
    merged_df.to_csv(output_csv, index=False)


In [54]:
# 使用例:
if __name__ == "__main__":
    image_info_csv = 'output-images/image_info.csv'  # 画像情報が格納されたCSVファイルのパス
    pdf_folder = '../sample-pdf'  # PDFファイルが格納されているフォルダのパス
    pattern = r"【図 [0-9]+\-[0-9]+】"  # 図番号の正規表現パターン
    output_csv = 'merged_info.csv'  # 出力CSVファイルのパス
    merge_image_and_figure_info(image_info_csv, pdf_folder, pattern, output_csv)
